<a href="https://colab.research.google.com/github/G-Gaddu/Quant-Material/blob/main/Portfolio_Optimisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install pandas-datareader
! pip install PyPortfolioOpt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 1.9 MB/s eta 0:00:00


In [16]:
# Import the necessary packages
import pandas_datareader.data as pdata
import datetime
import pandas as pd
from functools import reduce
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
from pypfopt import HRPOpt
import yfinance as yf

In [26]:
import warnings
warnings.filterwarnings('ignore')


In [6]:
# Limit the display limits on the rows and the columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [22]:
# Create a function that will pull the date for several stocks
def get_stock(ticker):
  data = yf.download(f"{ticker}",start="2022-01-01",end="2023-12-31")
  data[f'{ticker}'] = data["Close"]
  data = data[[f'{ticker}']]
  print(data.head())
  return data

In [23]:
# Create another function that takes a list of stocks and generates a single dataframe of stocks
def combine_stocks(tickers):
  data_frames = []
  for i in tickers:
    data_frames.append(get_stock(i))
  df_merged = reduce(lambda left,right: pd.merge(left,right,on=["Date"],how="outer"),data_frames)
  print(df_merged.head())
  return df_merged

stocks = ["AAPL", "AZN.L", "JPM", "ACN", "CCH.L", "BA", "WMT", "SHEL.L", "VTI", "GSK.L"]
portfolio = combine_stocks(stocks)


[*********************100%***********************]  1 of 1 completed


                  AAPL
Date                  
2022-01-03  182.009995
2022-01-04  179.699997
2022-01-05  174.919998
2022-01-06  172.000000
2022-01-07  172.169998


[*********************100%***********************]  1 of 1 completed


             AZN.L
Date              
2022-01-04  8473.0
2022-01-05  8460.0
2022-01-06  8356.0
2022-01-07  8399.0
2022-01-10  8334.0


[*********************100%***********************]  1 of 1 completed


...

                  WMT
Date                 
2022-01-03  48.216667
2022-01-04  47.333332
2022-01-05  47.973331
2022-01-06  47.840000
2022-01-07  48.296665
                 SHEL.L
Date                   
2022-01-04  1700.800049
2022-01-05  1722.199951
2022-01-06  1718.199951
2022-01-07  1736.599976
2022-01-10  1730.599976


[*********************100%***********************]  1 of 1 completed


                   VTI
Date                  
2022-01-03  242.970001
2022-01-04  242.509995
2022-01-05  237.250000
2022-01-06  237.190002
2022-01-07  236.110001


[*********************100%***********************]  1 of 1 completed

                  GSK.L
Date                   
2022-01-04  1609.153198
2022-01-05  1616.809082
2022-01-06  1600.489868
2022-01-07  1610.160645
2022-01-10  1623.256226
                  AAPL   AZN.L         JPM         ACN   CCH.L          BA  \
Date                                                                         
2022-01-03  182.009995     NaN  161.699997  407.209991     NaN  207.860001   
2022-01-04  179.699997  8473.0  167.830002  404.299988  2645.0  213.630005   
2022-01-05  174.919998  8460.0  163.779999  397.179993  2645.0  213.070007   
2022-01-06  172.000000  8356.0  165.520004  378.000000  2617.0  211.339996   
2022-01-07  172.169998  8399.0  167.160004  370.750000  2614.0  215.500000   

                  WMT       SHEL.L         VTI        GSK.L  
Date                                                         
2022-01-03  48.216667          NaN  242.970001          NaN  
2022-01-04  47.333332  1700.800049  242.509995  1609.153198  
2022-01-05  47.973331  1722.199951  2

In [24]:
# Save the prices to a csv file
portfolio.to_csv("portfolio.csv", index=False)
portfolio = pd.read_csv("portfolio.csv")

In [27]:
# Conduct Markowitz optimisation, find the mean historical return and the covariance matrix
mu = mean_historical_return(portfolio)
S = CovarianceShrinkage(portfolio).ledoit_wolf()

In [29]:
mu, S

(AAPL      0.027987
 AZN.L     0.094225
 JPM       0.025190
 ACN      -0.070487
 CCH.L    -0.088724
 BA        0.117608
 WMT       0.043182
 SHEL.L    0.229040
 VTI      -0.011696
 GSK.L    -0.059367
 dtype: float64,
             AAPL     AZN.L       JPM       ACN     CCH.L        BA       WMT  \
 AAPL    0.084786  0.003794  0.027539  0.043970  0.009919  0.040303  0.014701   
 AZN.L   0.003794  0.092991  0.005266  0.005145 -0.015934  0.005561  0.000740   
 JPM     0.027539  0.005266  0.070971  0.028622  0.012623  0.034602  0.008770   
 ACN     0.043970  0.005145  0.028622  0.080463  0.010641  0.038158  0.013884   
 CCH.L   0.009919 -0.015934  0.012623  0.010641  0.127601  0.016568  0.004304   
 BA      0.040303  0.005561  0.034602  0.038158  0.016568  0.132508  0.010821   
 WMT     0.014701  0.000740  0.008770  0.013884  0.004304  0.010821  0.056531   
 SHEL.L  0.002939  0.040413  0.012328  0.002524 -0.045054  0.008154  0.000230   
 VTI     0.037702  0.004286  0.026926  0.035646  0.008

In [30]:
# Determine the efficient frontier and weights
ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
print(dict(cleaned_weights))

{'AAPL': 0.0, 'AZN.L': 0.06582, 'JPM': 0.0, 'ACN': 0.0, 'CCH.L': 0.0, 'BA': 0.25473, 'WMT': 0.11601, 'SHEL.L': 0.56344, 'VTI': 0.0, 'GSK.L': 0.0}


In [31]:
# Display the portfolio performance
ef.portfolio_performance(verbose=True)

Expected annual return: 17.0%
Annual volatility: 24.8%
Sharpe Ratio: 0.61


(0.17021995539967705, 0.24801787719105184, 0.6056819657558814)

In [32]:
# Convert Weights into allocations from an investment of $100,000
latest_prices = get_latest_prices(portfolio)
da = DiscreteAllocation(weights, latest_prices, total_portfolio_value=100000)
allocation, leftover = da.greedy_portfolio()
print("Discrete allocation:", allocation)
print("Funds remaining: ${:.2f}".format(leftover))

Discrete allocation: {'SHEL.L': 21, 'BA': 97, 'WMT': 220}
Funds remaining: $8848.98


In [35]:
# Hierarchical Risk Parity
# Determine Returns
returns = portfolio.pct_change().dropna()

In [36]:
# Run the optimsation algorithm to determine the weights
hrp = HRPOpt(returns)
hrp_weights = hrp.optimize()

In [38]:
# Print the performance and weights
hrp.portfolio_performance(verbose=True)
print(dict(hrp_weights))

Expected annual return: 7.4%
Annual volatility: 13.6%
Sharpe Ratio: 0.40
{'AAPL': 0.03377793600801006, 'ACN': 0.06425590267549505, 'AZN.L': 0.11402077621321878, 'BA': 0.041586974872649125, 'CCH.L': 0.10120456683331265, 'GSK.L': 0.09415998597385376, 'JPM': 0.09185240815863345, 'SHEL.L': 0.09350161914661205, 'VTI': 0.07028202706025986, 'WMT': 0.2953578030579552}


In [39]:
# Determine discrete allocation using the weights
da_hrp = DiscreteAllocation(hrp_weights, latest_prices, total_portfolio_value=100000)
allocation, leftover = da_hrp.greedy_portfolio()
print("Discrete allocation (HRP):", allocation)
print("Funds remaining (HRP): ${:.2f}".format(leftover))

Discrete allocation (HRP): {'WMT': 562, 'AZN.L': 1, 'CCH.L': 5, 'GSK.L': 6, 'SHEL.L': 4, 'JPM': 54, 'VTI': 30, 'ACN': 18, 'BA': 16, 'AAPL': 18}
Funds remaining (HRP): $218.42


In [40]:
# Mean conditional value at risk
from pypfopt.efficient_frontier import EfficientCVaR

In [41]:
# Calculate weights and get performance
S = portfolio.cov()
ef_cvar = EfficientCVaR(mu, S)
cvar_weights = ef_cvar.min_cvar()
cleaned_weights = ef_cvar.clean_weights()
print(dict(cleaned_weights))

{'AAPL': 0.0, 'AZN.L': 0.0, 'JPM': 0.0, 'ACN': 0.0, 'CCH.L': 0.17539, 'BA': 0.0, 'WMT': 0.0, 'SHEL.L': 0.39884, 'VTI': 0.0, 'GSK.L': 0.42578}


In [42]:
# Get the discrete allocation
da_cvar = DiscreteAllocation(cvar_weights, latest_prices, total_portfolio_value=100000)
allocation, leftover = da_cvar.greedy_portfolio()
print("Discrete allocation (CVaR):", allocation)
print("Funds remaining (CVaR): ${:.2f}".format(leftover))

Discrete allocation (CVaR): {'GSK.L': 30, 'SHEL.L': 15, 'CCH.L': 8, 'ACN': 1, 'BA': 1}
Funds remaining (CVaR): $448.43
